### Zelle 1 — Imports & Reproduzierbarkeit (Seeds)

**Was macht diese Zelle?**  
Lädt alle Bibliotheken, die wir für Split und Preprocessing brauchen, und setzt einen festen Zufalls-Seed.

**Wie funktioniert das?**
- `numpy` und `pandas` sind Basiswerkzeuge für numerische Daten und Tabellen.
- `train_test_split` (scikit-learn) erzeugt den Train/Test-Split.
- `StandardScaler` skaliert numerische Features auf vergleichbare Skalen.
- `RANDOM_STATE` + `np.random.seed(...)` sorgen dafür, dass zufällige Prozesse (z. B. Split) reproduzierbar bleiben.

**Warum ist das wichtig?**
- Ohne Seed bekommst du bei jedem Run andere Splits → andere Ergebnisse → schwer zu debuggen und nicht audit-fähig.
- Reproduzierbarkeit ist eine Grundanforderung in ML-Engineering.


In [11]:
# ZELLE 1 — Imports + Reproduzierbarkeit (Seeds)
# Zweck: Einheitliche Ausgangslage für Split/Preprocessing, damit Ergebnisse reproduzierbar sind.

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
    

### Zelle 2 — Daten laden & Basiskontrolle

**Was macht diese Zelle?**  
Lädt das CSV-Dataset in einen DataFrame und macht eine erste Sichtprüfung.

**Wie funktioniert das?**
- `pd.read_csv(path)` liest die CSV-Datei ein.
- `df.shape` zeigt Zeilen/Spalten und ist der schnellste Plausibilitätscheck.
- `df.head()` zeigt die ersten Zeilen, um Spaltennamen und Wertebereiche zu prüfen.

**Warum ist das wichtig?**
- Frühzeitiges Erkennen von falschen Dateipfaden, falscher Dataset-Version oder Schema-Problemen.
- Ohne diesen Check können spätere Schritte „scheinbar“ laufen, aber fachlich auf falschen Daten beruhen.


In [12]:
# ZELLE 2 — Daten laden (wie bisher)
# Zweck: Rohdaten in DataFrame laden und kurz prüfen.

path = "../data/raw/creditcard.csv"
df = pd.read_csv(path)

print("Shape:", df.shape)
display(df.head())


Shape: (284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


### Zelle 3 — Features (X) und Zielvariable (y) trennen

**Was macht diese Zelle?**  
Teilt den Datensatz in:
- `X`: alle Feature-Spalten (Input fürs Modell)
- `y`: die Zielvariable `Class` (Label, das wir vorhersagen wollen)

**Wie funktioniert das?**
- `df.drop(columns=[target_col])` entfernt die Zielspalte aus X.
- `df[target_col]` extrahiert die Labels.
- `y.mean()` entspricht bei 0/1-Labels der Fraud-Rate.

**Warum ist das wichtig?**
- Saubere Trennung verhindert, dass das Modell die Zielvariable direkt „sehen“ kann (Leakage).
- Fraud-Rate ist der Kontext für alle späteren Entscheidungen zu Metriken und Thresholds.


In [13]:
# ZELLE 3 — Features (X) und Label (y) trennen
# Zweck: X enthält nur Features, y enthält nur Zielvariable (Class).

target_col = "Class"
X = df.drop(columns=[target_col])
y = df[target_col]

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Fraud rate (%):", (y.mean() * 100).round(4))


X shape: (284807, 30)
y shape: (284807,)
Fraud rate (%): 0.1727


### Zelle 4 — Train/Test Split (stratifiziert & deterministisch)

**Was macht diese Zelle?**  
Teilt die Daten in Trainings- und Testdaten:
- Train: Modell darf daraus lernen
- Test: bleibt unberührt bis zur finalen Evaluation

**Wie funktioniert das?**
- `test_size=0.2`: 80/20-Split
- `stratify=y`: sorgt dafür, dass der Anteil Fraud in Train und Test ähnlich bleibt
- `random_state=RANDOM_STATE`: macht den Split reproduzierbar

**Warum ist das wichtig?**
- Ohne Stratify kann es passieren, dass im Testset zu wenige Fraud-Fälle landen → Evaluation wird instabil.
- Ohne festen Seed sind Ergebnisse nicht reproduzierbar.
- Der Testset ist dein „Audit-Check“: Er darf nicht in Entscheidungen einfließen.


In [14]:
# ZELLE 4 — Train/Test Split (stratifiziert, deterministisch)
# Zweck: Testdaten bleiben "unseen". Stratify sorgt für ähnlichen Fraud-Anteil in Train/Test.

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

print("Train shape:", X_train.shape, " Test shape:", X_test.shape)
print("Train fraud rate (%):", (y_train.mean() * 100).round(4))
print("Test  fraud rate (%):", (y_test.mean() * 100).round(4))


Train shape: (227845, 30)  Test shape: (56962, 30)
Train fraud rate (%): 0.1729
Test  fraud rate (%): 0.172


### Zelle 5 — Feature-Gruppen definieren (Scaling vs. nicht Scaling)

**Was macht diese Zelle?**  
Definiert explizit, welche Spalten skaliert werden:
- `Time` und `Amount` werden skaliert
- `V1–V28` bleiben unverändert

**Wie funktioniert das?**
- `scale_cols = ["Time", "Amount"]`
- Guard: prüft, ob diese Spalten überhaupt existieren
- `other_cols` sammelt alle restlichen Features (meist PCA-Features)

**Warum ist das wichtig?**
- Tabellarische ML-Pipelines scheitern oft daran, dass Features „blind“ transformiert werden.
- Hier machen wir die Entscheidung explizit und nachvollziehbar.
- Dadurch bleibt die Pipeline auditierbar: man sieht sofort, was warum skaliert wird.


In [15]:
# ZELLE 5 — Feature-Gruppen definieren
# Zweck: Nur Time und Amount skalieren. V1–V28 bleiben unverändert.

scale_cols = ["Time", "Amount"]

# Optionaler Guard: Prüfen, ob die Spalten existieren
missing = [c for c in scale_cols if c not in X_train.columns]
if missing:
    raise ValueError(f"Missing expected columns for scaling: {missing}")

# Alle übrigen Features (PCA-Features + evtl. weitere)
other_cols = [c for c in X_train.columns if c not in scale_cols]

print("Columns to scale:", scale_cols)
print("Other columns count:", len(other_cols))


Columns to scale: ['Time', 'Amount']
Other columns count: 28


### Zelle 6 — Scaling (FIT nur auf Train, TRANSFORM auf Train & Test)

**Was macht diese Zelle?**  
Skaliert `Time` und `Amount` leakage-sicher.

**Wie funktioniert das?**
- `scaler.fit(X_train[scale_cols])` lernt Mittelwert und Standardabweichung **nur aus Train**
- `scaler.transform(...)` wendet diese Parameter auf Train und Test an
- Wir arbeiten auf `.copy()`, damit wir die Originaldaten nicht zerstören

**Warum ist das wichtig?**
- Wenn du auf Gesamtdata fitten würdest, „kennt“ der Scaler Informationen aus dem Testset → Data Leakage.
- Leakage führt fast immer zu zu guten Metriken und falschen Schlussfolgerungen.
- Das ist eine der häufigsten Fehlerquellen in ML-Projekten.


In [16]:
# ZELLE 6 — Scaler NUR auf Train fitten (Leakage-Schutz)
# Zweck: StandardScaler lernt Mittelwert/Std nur aus Train-Daten.

scaler = StandardScaler()
scaler.fit(X_train[scale_cols])  # <-- FIT nur auf TRAIN!

# Transform auf Train und Test
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[scale_cols] = scaler.transform(X_train[scale_cols])
X_test_scaled[scale_cols]  = scaler.transform(X_test[scale_cols])


### Zelle 7 — Sanity Checks nach dem Scaling

**Was macht diese Zelle?**  
Überprüft, ob die Skalierung technisch korrekt war.

**Wie funktioniert das?**
- Check auf NaNs: Skalierung darf keine fehlenden Werte erzeugen
- Statistiken auf `Time` und `Amount`:
  - Train: mean ~ 0 und std ~ 1 (weil darauf gefittet wurde)
  - Test: nicht zwingend exakt 0/1, aber sinnvoll skaliert

**Warum ist das wichtig?**
- Ohne Checks merkst du Fehler erst viel später (z. B. wenn Modelle komisch reagieren).
- Diese Zelle ist eine technische Qualitätskontrolle der Pipeline.


In [17]:
# ZELLE 7 — Quick Sanity Checks nach Scaling
# Zweck: Prüfen, ob Scaling korrekt war und keine NaNs entstanden sind.

# 1) Keine NaNs
print("NaNs in X_train_scaled:", X_train_scaled.isna().sum().sum())
print("NaNs in X_test_scaled:", X_test_scaled.isna().sum().sum())

# 2) Train: Time/Amount sollten ~0 mean und ~1 std haben (nicht exakt, aber nahe)
train_stats = X_train_scaled[scale_cols].agg(["mean", "std"]).T
display(train_stats)

# 3) Test: nicht exakt 0/1 (weil mit Train-Parametern transformiert), aber sinnvoll skaliert
test_stats = X_test_scaled[scale_cols].agg(["mean", "std"]).T
display(test_stats)


NaNs in X_train_scaled: 0
NaNs in X_test_scaled: 0


,mean,std
Time,-1.407707e-16,1.000002
Amount,-2.020811e-17,1.000002


,mean,std
Time,-0.007500,0.999960
Amount,0.003456,0.987933


### Zelle 8 — Final Output der Preprocessing-Stage

**Was macht diese Zelle?**  
Gibt die finalen, vorverarbeiteten Datensätze aus, die wir ab jetzt überall verwenden.

**Wie funktioniert das?**
- `X_train_proc` und `X_test_proc` sind die standardisierten Feature-Matrizen
- Labels bleiben unverändert: `y_train`, `y_test`
- Kurzer Check: Shapes und Fraud-Rates

**Warum ist das wichtig?**
- Ab jetzt soll kein späterer Schritt mehr auf `df` oder `X_train_scaled` herumarbeiten.
- Einheitliche, stabile Variablennamen reduzieren Fehler und machen den Workflow klar.


In [18]:
# ZELLE 8 — Final Output: Preprocessed Datasets bereitstellen
# Zweck: Einheitliche Variablennamen, die wir in Baselines und NN wiederverwenden.

X_train_proc = X_train_scaled
X_test_proc = X_test_scaled

print("X_train_proc shape:", X_train_proc.shape)
print("X_test_proc shape:", X_test_proc.shape)
print("y_train fraud rate (%):", (y_train.mean() * 100).round(4))
print("y_test  fraud rate (%):", (y_test.mean() * 100).round(4))


X_train_proc shape: (227845, 30)
X_test_proc shape: (56962, 30)
y_train fraud rate (%): 0.1729
y_test  fraud rate (%): 0.172


### Zelle 9 — (Optional) Spaltenreihenfolge fixieren

**Was macht diese Zelle?**  
Stellt sicher, dass Train und Test exakt dieselbe Spaltenreihenfolge haben.

**Wie funktioniert das?**
- `reindex(sorted(...))` sortiert die Spalten im Trainset deterministisch
- Testset wird exakt auf diese Spaltenreihenfolge gebracht
- `assert` stellt sicher, dass es wirklich identisch ist

**Warum ist das wichtig?**
- Manche Modelle (und besonders später Export/Inference) erwarten exakt gleiche Feature-Reihenfolge.
- Das ist ein klassischer Engineering-Bug: gleiche Spalten, aber andere Reihenfolge → falsche Predictions.
- Diese Zelle macht die Pipeline robust.


In [19]:
# ZELLE 9 — (Optional) Reihenfolge der Spalten fixieren (Engineering-Schutz)
# Zweck: Sicherstellen, dass Train und Test exakt gleiche Spaltenreihenfolge haben.

X_train_proc = X_train_proc.reindex(sorted(X_train_proc.columns), axis=1)
X_test_proc  = X_test_proc.reindex(X_train_proc.columns, axis=1)

assert list(X_train_proc.columns) == list(X_test_proc.columns)
print("Column order aligned. Feature count:", X_train_proc.shape[1])


Column order aligned. Feature count: 30


### Zelle 10 — (Optional) Artefakte speichern (Scaler)

**Was macht diese Zelle?**  
Speichert den train-gefitteten Scaler als Artefakt, damit wir dieselbe Transformation später wiederverwenden können.

**Wie funktioniert das?**
- `joblib.dump(scaler, "../models/scaler.pkl")` speichert das Objekt effizient auf Disk
- Das `models/`-Verzeichnis ist bewusst per `.gitignore` ausgeschlossen

**Warum ist das wichtig?**
- Für echte ML-Systeme musst du Preprocessing bei Training, Evaluation und späterer Inferenz identisch anwenden.
- Der Scaler ist Teil des Modells (Pipeline-Artefakt).
- Ohne Speichern würdest du beim nächsten Run evtl. andere Parameter lernen, was Vergleiche erschwert.


In [20]:
# ZELLE 10 — (Optional) Artefakte speichern (nicht in Git)
# Zweck: Scaler später für Evaluation/Explainability/Inference wiederverwenden.
# Hinweis: models/ ist per .gitignore ausgeschlossen.

import joblib
joblib.dump(scaler, "../models/scaler.pkl")
print("Saved scaler to models/scaler.pkl")


Saved scaler to models/scaler.pkl
